<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/her_zaman_dilimide_sinyaller_telegrama_g%C3%B6nderme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
from tradingview_ta import TA_Handler, Interval, Exchange
import requests

# Telegram bot bilgilerinizi girin
BOT_TOKEN = 'BOT_TOKEN'
CHAT_ID = 'CHAT_ID'

def send_telegram_message(message):
    url = f"https://api.telegram.org/bot(BOT_TOKEN)/sendMessage"
    payload = {"chat_id": CHAT_ID, "text": message, "parse_mode": "Markdown"}
    requests.post(url, data=payload)

# Son gönderim zamanlarını saklayacak sözlük
last_sent_time = {}

def check_conditions():
    coins = [
        "BTCUSDT.P", "ETHUSDT.P", "SOLUSDT.P", "TONUSDT.P", "DOGEUSDT.P",
        "XRPUSDT.P", "APEUSDT.P", "PEPEUSDT.P", "UXLINKUSDT.P", "ALPHAUSDT.P",
        "1INCHUSDT.P", "AAVEUSDT.P", "ACEUSDT.P", "ACHUSDT.P", "ADAUSDT.P",
        "AEVOUSDT.P", "AGLDUSDT.P", "AIDOGEUSDT.P", "ALGOUSDT.P", "API3USDT.P",
        "APTUSDT.P", "ARUSDT.P", "ARBUSDT.P", "ATHUSDT.P", "ATOMUSDT.P",
        "AUCTIONUSDT.P", "AVAXUSDT.P", "AXSUSDT.P", "BADGERUSDT.P", "BALUSDT.P",
        "BANDUSDT.P", "BATUSDT.P", "BCHUSDT.P", "BICOUSDT.P", "BIGTIMEUSDT.P",
        "BLURUSDT.P", "BNBUSDT.P", "BNTUSDT.P", "BOMEUSDT.P", "BONKUSDT.P",
        "BONEUSDT.P", "BRETTUSDT.P", "BSVUSDT.P", "CATUSDT.P", "CATIUSDT.P",
        "CELOUSDT.P", "CETUSUSDT.P", "CFXUSDT.P", "CHZUSDT.P", "COMPUSDT.P",
        "COREUSDT.P", "CROUSDT.P", "CRVUSDT.P", "CSPRUSDT.P", "CTCUSDT.P",
        "CVCUSDT.P", "CVXUSDT.P", "DGBUSDT.P", "DOGSUSDT.P", "DOTUSDT.P",
        "DYDXUSDT.P", "EGLDUSDT.P", "EIGENUSDT.P", "ENJUSDT.P", "ENSUSDT.P",
        "EOSUSDT.P", "ETCUSDT.P", "ETHWUSDT.P", "ETHFIUSDT.P", "FILUSDT.P",
        "FLMUSDT.P", "FLOKIUSDT.P", "FLOWUSDT.P", "FOXYUSDT.P", "FTMUSDT.P",
        "FXSUSDT.P", "GALAUSDT.P", "GASUSDT.P", "GFTUSDT.P", "GLMUSDT.P",
        "GMTUSDT.P", "GMXUSDT.P", "GODSUSDT.P", "GRTUSDT.P", "HBARUSDT.P",
        "HMSTRUSDT.P", "ICPUSDT.P", "ICXUSDT.P", "IDUSDT.P", "IMXUSDT.P",
        "INJUSDT.P", "IOSTUSDT.P", "IOTAUSDT.P", "JOEUSDT.P", "JSTUSDT.P",
        "JTOUSDT.P", "JUPUSDT.P", "KNCUSDT.P", "KSMUSDT.P", "LDOUSDT.P",
        "LINKUSDT.P", "LOOKSUSDT.P", "LPTUSDT.P", "LQTYUSDT.P", "LRCUSDT.P",
        "LSKUSDT.P", "LTCUSDT.P", "LUNAUSDT.P", "LUNCUSDT.P", "MAGICUSDT.P",
        "MANAUSDT.P", "MASKUSDT.P", "MAXUSDT.P", "MEMEUSDT.P", "MERLUSDT.P",
        "METISUSDT.P", "MEWUSDT.P", "MINAUSDT.P", "MKRUSDT.P", "MOODENGUSDT.P",
        "MOVRUSDT.P", "NEARUSDT.P", "NEIROETHUSDT.P", "NEIROUSDT.P", "NEOUSDT.P",
        "NMRUSDT.P", "NOTUSDT.P", "OMUSDT.P", "ONDOUSDT.P", "ONEUSDT.P",
        "ONTUSDT.P", "OPUSDT.P", "ORBSUSDT.P", "ORDIUSDT.P", "PEOPLEUSDT.P",
        "PERPUSDT.P", "POLUSDT.P", "POPCATUSDT.P", "PRCLUSDT.P", "PUFFERUSDT.P",
        "PYTHUSDT.P", "QTUMUSDT.P", "RACAUSDT.P", "RAYUSDT.P", "RDNTUSDT.P",
        "RENDERUSDT.P", "RONUSDT.P", "RSRUSDT.P", "RVNUSDT.P", "SANDUSDT.P",
        "SHIBUSDT.P", "SLPUSDT.P", "SNXUSDT.P", "SSVUSDT.P", "STORJUSDT.P",
        "STRKUSDT.P", "STXUSDT.P", "SUIUSDT.P", "SUSHIUSDT.P", "SWEATUSDT.P",
        "TUSDT.P", "TAOUSDT.P", "THETAUSDT.P", "TIAUSDT.P", "TNSRUSDT.P",
        "TRBUSDT.P", "TRXUSDT.P", "TURBOUSDT.P", "ULTIUSDT.P", "UMAUSDT.P",
        "UNIUSDT.P", "USTCUSDT.P", "VELOUSDT.P", "VRAUSDT.P", "WUSDT.P",
        "WAXPUSDT.P", "WIFUSDT.P", "WLDUSDT.P", "WOOUSDT.P", "XCHUSDT.P",
        "XLMUSDT.P", "XTZUSDT.P", "YFIUSDT.P", "YGGUSDT.P", "ZENTUSDT.P",
        "ZETAUSDT.P", "ZILUSDT.P", "ZKUSDT.P", "ZROUSDT.P", "ZRXUSDT.P",
    ]

    # Kullanmak istediğiniz zaman dilimleri
    intervals = [
        Interval.INTERVAL_1_MINUTE,
        Interval.INTERVAL_5_MINUTES,
        Interval.INTERVAL_15_MINUTES,
        Interval.INTERVAL_30_MINUTES,
        Interval.INTERVAL_1_HOUR
    ]

    for coin in coins:
        for interval in intervals:
            try:
                handler = TA_Handler(
                    symbol=coin,
                    exchange="OKX",
                    screener="crypto",
                    interval=interval
                )

                analysis = handler.get_analysis()

                # RSI değerini al
                rsi_value = analysis.indicators.get('RSI')
                previous_rsi_value = analysis.indicators.get('RSI[1]')

                if rsi_value is not None and previous_rsi_value is not None:
                    # RSI farkı 20 birimden fazla mı kontrol et
                    if abs(rsi_value - previous_rsi_value) >= 20:
                        # LONG veya SHORT sinyaline göre anahtar belirle
                        direction = "LONG" if rsi_value > previous_rsi_value else "SHORT"
                        message_key = f"{coin}_{interval}_{direction}"

                        # Son mesaj zamanını kontrol et
                        current_time = time.time()
                        if message_key not in last_sent_time or (current_time - last_sent_time[message_key] > 1800):
                            if direction == "LONG":
                                send_telegram_message(f"*{coin}* *(C20L)* *(LONG)*🚀  ({interval}): {previous_rsi_value:.2f} -> {rsi_value:.2f}")
                            else:
                                send_telegram_message(f"*{coin}* *(C20S)* *(SHORT)* 🔻 ({interval}): {previous_rsi_value:.2f} -> {rsi_value:.2f}")

                            # Mesaj gönderildiği zamanı kaydet
                            last_sent_time[message_key] = current_time

            except Exception as e:
                print(f"Hata oluştu: {coin}, Zaman Dilimi: {interval}, Hata: {e}")

def main():
    while True:
        check_conditions()
        # Her dakika bir kontrol eder
        time.sleep(60)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'tradingview_ta'